In [0]:
import pandas as pd
import numpy as np
from unidecode import unidecode
from sklearn.tree import DecisionTreeRegressor
from functools import reduce
from databricks.feature_engineering import FeatureEngineeringClient
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException

In [0]:
def normalize_columns(df):
    df.columns = [unidecode(col).strip().lower() for col in df.columns]
    return df

def normalize_text_values(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].apply(lambda x: unidecode(x).strip().lower() if isinstance(x, str) else x)
    return df

In [0]:
def features_numericas_combinadas(df, numericas):
    aux = pd.DataFrame()
    for i in range(len(numericas)):
        for j in range(i + 1, len(numericas)):
            col1, col2 = numericas[i], numericas[j]
            aux[f'{col1}_x_{col2}'] = df[col1] * df[col2]
            aux[f'{col1}_div_{col2}'] = df[col1] / df[col2].replace(0, np.nan)
            aux[f'{col2}_div_{col1}'] = df[col2] / df[col1].replace(0, np.nan)
    return pd.concat([df.matricula, aux], axis=1)

def features_binarias_numericas_combinadas(df, binarias, numericas):
    aux = pd.DataFrame()
    for binaria in binarias:
        for num in numericas:
            aux[f'{binaria}_x_{num}'] = df[binaria] * df[num]
    return pd.concat([df.matricula, aux], axis=1)

def safe_str_convert(val):
    return str(val) if pd.notnull(val) else np.nan

def features_binarias_categoricas_combinadas(df, binarias, categoricas):
    aux = pd.DataFrame()
    for binaria in binarias:
        for cat in categoricas:
            aux[f'{binaria}_x_{cat}'] = df[binaria] * df[cat]
    aux = aux.applymap(safe_str_convert)
    return pd.concat([df.matricula, aux], axis=1)

def features_binarias_combinadas(df, binarias):
    aux = pd.DataFrame()
    for i in range(len(binarias)):
        for j in range(i + 1, len(binarias)):
            aux[f'{binarias[i]}_x_{binarias[j]}'] = df[binarias[i]] * df[binarias[j]]
    aux = aux.applymap(safe_str_convert)
    return pd.concat([df.matricula, aux], axis=1)

def features_binning(df, idade_bins):
    aux = pd.DataFrame()

    aux['cat_fumante'] = pd.cut(
        df.fumante,
        bins=[-np.inf, 0, 1],
        labels=['nao', 'sim']
    )
    aux['cat_facebook'] = pd.cut(
        df.facebook,
        bins=[-np.inf, 0, 1],
        labels=['nao', 'sim']
    )
    aux['cat_idade'] = pd.cut(
        df.idade,
        bins=[-np.inf]+idade_bins.Upper_Bound.to_list(),
        labels=['jovem', 'jovem-adulto', 'adulto', 'senior']
    )
    aux['cat_imc'] = pd.cut(
        df['imc'],
        bins=[-float('inf'), 16, 16.9, 18.4, 24.9, 29.9, 34.9, 39.9, float('inf')],
        labels=['magreza grave', 'magreza moderada', 'magreza leve', 
                'normal', 'sobrepeso', 'obesidade grau 1', 'obesidade severa', 
                'obesidade mórbida']
    )
    aux['cat_filhos'] = pd.cut(
        df.filhos,
        bins=[-np.inf, 0, 2, 3, 4],
        labels=['nenhum', 'poucos', 'alguns', 'muitos']
    )
    aux['cat_regiao'] = df['regiao'].apply(safe_str_convert)
    aux['cat_classe'] = df['classe'].apply(safe_str_convert)

    return pd.concat([df.matricula, aux], axis=1)

In [0]:
def categorize_regression_bins(df, n_bins, target, numeric_var):
    df = df.dropna(subset=[numeric_var, target])
    
    X = df[[numeric_var]]
    y = df[target]

    # Árvore rasa
    tree = DecisionTreeRegressor(max_leaf_nodes=n_bins, random_state=42)
    tree.fit(X, y)

    leaf_ids = tree.apply(X)
    temp_df = pd.DataFrame({
        numeric_var: df[numeric_var],
        "Leaf": leaf_ids,
        target: y
    })

    result = temp_df.groupby("Leaf").agg({
        target: "mean",
        numeric_var: ["count", "min", "max"]
    }).reset_index()

    result.columns = ["Leaf", "Gasto_Medio", "Volume", "Lower_Bound", "Upper_Bound"]

    result = result.sort_values(by="Lower_Bound").reset_index(drop=True)

    # Corrigir sobreposição entre os limites
    for i in range(1, len(result)):
        result.at[i, "Lower_Bound"] = max(result.at[i, "Lower_Bound"], result.at[i - 1, "Upper_Bound"])

    # Ajustar extremos
    result.at[0, "Lower_Bound"] = -np.inf
    result.at[len(result) - 1, "Upper_Bound"] = np.inf

    return result

In [0]:
df = pd.read_excel('data/Seguro Saúde - Modelagem.xlsx', sheet_name='MODELAGEM')

df = normalize_columns(df)
df = normalize_text_values(df)

y = df[['matricula', 'valor']]

df = df.drop(columns=['nascimento', 'valor'])

display(df.head())
display(y.head())

In [0]:
# Detalhando as variáveis
numericas = ['idade', 'imc', 'filhos']
binarias = ['fumante', 'facebook'] # Na verdade, numérica 
categoricas = ['regiao', 'classe'] # Na verdade, numérica
target = 'valor'

df_base = df[['matricula'] + numericas + binarias + categoricas + ['sexo', 'signo']].copy()

In [0]:
df_feat1 = features_numericas_combinadas(df_base.copy(), numericas)
display(df_feat1.head())

In [0]:
df_feat2 = features_binarias_numericas_combinadas(df_base.copy(), binarias, numericas)
display(df_feat2.head())

In [0]:
df_feat3 = features_binarias_categoricas_combinadas(df_base.copy(), binarias, categoricas)
display(df_feat3.head())

In [0]:
df_feat4 = features_binarias_combinadas(df_base.copy(), binarias)
display(df_feat4.head())

In [0]:
aux = pd.concat([df_base.copy(), y.valor], axis=1)
idade_bins = categorize_regression_bins(aux, n_bins=4, target='valor', numeric_var='idade')
display(idade_bins)

In [0]:
df_feat5 = features_binning(df_base.copy(), idade_bins)
display(df_feat5.head())

In [0]:
display(df_base.head())

In [0]:
df_base = df_base.drop(columns=['fumante', 'facebook', 'regiao', 'classe'])

dfs = [df_base, df_feat1, df_feat2, df_feat3, df_feat4, df_feat5]

x_engineered = reduce(lambda left,right: pd.merge(left,right,on='matricula',how='inner'), dfs)

display(x_engineered.head())

In [0]:
x_engineered.to_csv("data/x_enginneered.csv", index=False)


# Tentando criar Feature Store

In [0]:
dfs = {
    'df_base': {
        'df': df_base,
        'table_name': 'fs_base',
        'description': 'Features originais'
    }, 
    'df_feat1': {
        'df': df_feat1,
        'table_name': 'fs_numericas_combinadas',
        'description': 'Features numéricas combinadas'
    }, 
    'df_feat2': {
        'df': df_feat2,
        'table_name': 'fs_binarias_numericas_combinadas',
        'description': 'Features binárias e numéricas combinadas'
    }, 
    'df_feat3': {
        'df': df_feat3,
        'table_name': 'fs_binarias_categoricas_combinadas',
        'description': 'Features binárias e categoricas combinadas'
    },
    'df_feat4': {
        'df': df_feat4,
        'table_name': 'fs_binarias_combinadas',
        'description': 'Features binárias combinadas'
    }, 
    'df_feat5': {
        'df': df_feat5,
        'table_name': 'fs_binning',
        'description': 'Features categorizadas e quantizadas'
    }
}


In [0]:
fe = FeatureEngineeringClient()
spark = SparkSession.getActiveSession()
# bucket_path = "s3a://databricks-accenture-case-workspace-stack-5e610-bucket/unity-catalog/feature_store/healthcare"
spark.sparkContext.setLogLevel("ERROR")
import traceback
import re

for df_name,df in dfs.items():
    print(f'Criando tabela do {df_name}')
    try:
        df_spark = spark.createDataFrame(df['df'])

        fe.create_table(
            name=f'feature_store.healthcare.{df["table_name"]}',
            df=df_spark,
            description=df['description'],
            primary_keys=['matricula'],
            schema=df_spark.schema
        )

        fe.write_table(
            name=f'feature_store.healthcare.{df["table_name"]}',
            df=df_spark,
            mode='merge'
        )

        print(f'Tabe;a {df["table_name"]} criada e escrita com sucesso')
    except AnalysisException as e:
        print(f'[Erro Spark] Falha ao criar a tabela {df["table_name"]}: {e}')
    except Exception as e:
        tb = traceback.format_exc().strip().split('\n')
        clean_msg = next((line for line in tb if not re.search(r'\.py|\.(scala|java):\d+', line)), str(e))
        print(f"Erro ao criar a tabela `{df['table_name']}`: {clean_msg}")


#Conjunto de Teste

In [0]:
df_test = pd.read_excel('data/Seguro Saúde - Teste Final.xlsx')

df_test = normalize_columns(df_test)
df_test = normalize_text_values(df_test)

y_test = df_test[['matricula', 'valor']]

df_test = df_test.drop(columns=['nascimento', 'valor'])

display(df_test.head())
display(y_test.head())

In [0]:
# Detalhando as variáveis
numericas = ['idade', 'imc', 'filhos']
binarias = ['fumante', 'facebook'] # Na verdade, numérica 
categoricas = ['regiao', 'classe'] # Na verdade, numérica
target = 'valor'

df_test_base = df_test[['matricula'] + numericas + binarias + categoricas + ['sexo', 'signo']].copy()

In [0]:
df_test_feat1 = features_numericas_combinadas(df_test_base.copy(), numericas)
display(df_test_feat1.head())


In [0]:
df_test_feat2 = features_binarias_numericas_combinadas(df_test_base.copy(), binarias, numericas)
display(df_test_feat2.head())

In [0]:
df_test_feat3 = features_binarias_categoricas_combinadas(df_test_base.copy(), binarias, categoricas)
display(df_test_feat3.head())

In [0]:
df_test_feat4 = features_binarias_combinadas(df_test_base.copy(), binarias)
display(df_test_feat4.head())

In [0]:
aux = pd.concat([df_test_base.copy(), y_test.valor], axis=1)
idade_bins = categorize_regression_bins(aux, n_bins=4, target='valor', numeric_var='idade')
display(idade_bins)

In [0]:
df_test_feat5 = features_binning(df_test_base.copy(), idade_bins)
display(df_test_feat5.head())

In [0]:
df_test_base = df_test_base.drop(columns=['fumante', 'facebook', 'regiao', 'classe'])

dfs_test = [df_test_base, df_test_feat1, df_test_feat2, df_test_feat3, df_test_feat4, df_test_feat5]

x_test_engineered = reduce(lambda left,right: pd.merge(left,right,on='matricula',how='inner'), dfs_test)

display(x_test_engineered.head())

In [0]:
x_test_engineered.to_csv("data/x_test_enginneered.csv", index=False)